# RFM Anlysis

## Hello Everyone,

Today I will show you a RFM anlysis practice. If you want to learn information about RFM anlysis you could check-out my medium article about it from here.

First things first, we need to learn about the Dataset which we will use today.

The name of the dataset is "online_retail_II", you can find the dataset from [here](https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci). Dataset includes all the transactions fro non-store online retail between 01/12/2009 and 09/12/2011. We will look at the transactions from 2010.

The dataset has 8 columns:
1. InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
2. StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
3. Description: Product (item) name. Nominal.
4. Quantity: The quantities of each product (item) per transaction. Numeric.
5. InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
6. UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).
7. CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
8. Country: Country name. Nominal. The name of the country where a customer resides.

So, let's start to practice!

In [3]:
# Reading the excel as a dataframe:
import pandas as pd
import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
df_ = pd.read_csv('online_retail_II.csv')
df_.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom


In [5]:
# Making a new dataframe from the data of 2010:
df_['InvoiceDate'] = pd.to_datetime(df_['InvoiceDate'])
df = df_[df_['InvoiceDate'].dt.year == 2010].copy()
df.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
45228,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0,United Kingdom
45229,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0,United Kingdom
45230,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0,United Kingdom
45231,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN,United Kingdom
45232,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN,United Kingdom
45233,493413,21723,ALPHABET HEARTS STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN,United Kingdom
45234,493414,21844,RETRO SPOT MUG,36,2010-01-04 10:28:00,2.55,14590.0,United Kingdom
45235,493414,21533,RETRO SPOT LARGE MILK JUG,12,2010-01-04 10:28:00,4.25,14590.0,United Kingdom
45236,493414,37508,NEW ENGLAND CERAMIC CAKE SERVER,2,2010-01-04 10:28:00,2.55,14590.0,United Kingdom
45237,493414,35001G,HAND OPEN SHAPE GOLD,2,2010-01-04 10:28:00,4.25,14590.0,United Kingdom


In [6]:
# Let's look at descriptive statics of the DataFrame
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,522714.0,10.243041,107.556003,-9600.00,1.00,3.0,10.00,19152.00
Price,522714.0,4.832495,151.689528,-53594.36,1.25,2.1,4.21,25111.09
Customer ID,412624.0,15362.985284,1686.096299,12346.00,13983.00,15311.0,16814.00,18287.00


As we can see, there is some negative values at the columns of "Quantity" and "Price". These negatives values are caused by returned products. So, let's get rid of from returned items and check the descriptive statics again:

In [7]:
df = df[df['Price']>0]
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,519007.0,10.683370,91.233775,-9360.000,1.00,3.0,10.00,19152.00
Price,519007.0,5.130113,104.794326,0.001,1.25,2.1,4.21,25111.09
Customer ID,412597.0,15363.083941,1686.059954,12346.000,13983.00,15311.0,16814.00,18287.00


In [8]:
# Checking if there is any null value:
df.isnull().sum()

Invoice             0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
Price               0
Customer ID    106410
Country             0
dtype: int64

As we can see, we have some null values at the column of Customer ID. So, get rid of them as well and check it again.

In [9]:
df.dropna(inplace=True)
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

## Now our DataFrame is ready for some RFM analysis...

First of all we need to define an analysis date to figure out recency values and it must be more recent than most recent order date. Therefore, let's have a look at the date of most recent order and define an analysis date according to it.

In [10]:
# Date of most recent order:
print(df['InvoiceDate'].max())

2010-12-23 16:06:00


As we can see most recent order date is 2010-12-09. So, we can set an anlysis date as 2010-12-11 (2 days after most recent order date).

In [11]:
today_date = dt.datetime(2010, 12, 11)

Now we are ready to define RFM metrics.

In [12]:
# Definin RFM metrics:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda invoice: invoice.nunique(),
                                     'Price': 'sum'}).reset_index()

# Naming rfm columns:
rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']
rfm.head(10)

,customer_id,recency,frequency,monetary
0,12346.0,67,10,559.36
1,12347.0,3,2,252.54
2,12348.0,-6,2,63.99
3,12349.0,43,3,875.34
4,12351.0,11,1,49.46
5,12352.0,11,2,54.60
6,12353.0,44,1,38.78
7,12355.0,203,1,52.78
8,12356.0,16,3,261.38
9,12357.0,24,2,1395.64


As we can see, it's a little bit hard to make a coprasion with these values. So, we need a standardization. We can standardize RFM values by creating 5-point scale which will includes 5 different value range.

In [13]:
# Making rfm scores on 5-point scale by using "qcut" method:
rfm['recency_score'] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['frequency_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[5, 4, 3, 2, 1])
rfm['monetary_score'] = pd.qcut(rfm['monetary'], 5, labels=[5, 4, 3, 2, 1])

rfm.head(10)

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score
0,12346.0,67,10,559.36,2,1,1
1,12347.0,3,2,252.54,5,4,2
2,12348.0,-6,2,63.99,5,4,4
3,12349.0,43,3,875.34,3,3,1
4,12351.0,11,1,49.46,4,5,4
5,12352.0,11,2,54.60,4,4,4
6,12353.0,44,1,38.78,3,5,5
7,12355.0,203,1,52.78,1,5,4
8,12356.0,16,3,261.38,4,3,2
9,12357.0,24,2,1395.64,4,4,1


Now we could make a comprasion very easily. But a segmentation is more useful than a raw comprasion. We could make a segmentation by using the table below

<img src="1_oUKMc3gk53hccFX4YNeu1g.png" />

First of all, let's define our RF Scores.

In [14]:
# Defining RF Score:
rfm['RF_SCORE'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)
rfm['RF_SCORE'].head(10)

0    21
1    54
2    54
3    33
4    45
5    44
6    35
7    15
8    43
9    44
Name: RF_SCORE, dtype: object

And last but not least, let's make the segmentation.

In [15]:
# Making the segmentation by mapping:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

## Here it is, our segmentated DataFrame !

In [16]:
rfm.head(10)

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
0,12346.0,67,10,559.36,2,1,1,21,hibernating
1,12347.0,3,2,252.54,5,4,2,54,champions
2,12348.0,-6,2,63.99,5,4,4,54,champions
3,12349.0,43,3,875.34,3,3,1,33,need_attention
4,12351.0,11,1,49.46,4,5,4,45,loyal_customers
5,12352.0,11,2,54.60,4,4,4,44,loyal_customers
6,12353.0,44,1,38.78,3,5,5,35,loyal_customers
7,12355.0,203,1,52.78,1,5,4,15,cant_loose
8,12356.0,16,3,261.38,4,3,2,43,potential_loyalists
9,12357.0,24,2,1395.64,4,4,1,44,loyal_customers


I can hear you say "ummmm... cool... now what?". Now we could make customer oriented decisions with these segments and we can use our limited resources as efficiently as possible!

## Let's give some decision example by usage of RFM analysis!

First, let's have a first look at the segments.

In [17]:
rfm.groupby('segment').agg({'frequency': 'sum',
                            'customer_id': 'count'})

,frequency,customer_id
segment,,
about_to_sleep,2271,312
at_Risk,1601,800
cant_loose,565,565
champions,164,113
hibernating,2018,318
loyal_customers,794,586
need_attention,571,205
new_customers,9084,458
potential_loyalists,3006,719


When we look at the "at_Risk" segment, we see that the most customers enter this segment and customers with low recency value fall into this segment. These indicators show that the "at_Risk" segment is the most important segment in terms of customer retention rate and customer churn rate.

The "new_customers" segment is another segment that should be given importance because of frequency and number of customers of it.

The 'potential_loyalists' segment, which is important in terms of the number of customers it contains and has an average value in terms of frequency, will be the last segment we will consider.

Now, we could make an analysis of chosen segments RFM scores.

In [18]:
rfm[(rfm['segment'] == 'at_Risk') | (rfm['segment'] == 'potential_loyalists') | (rfm['segment'] == 'new_customers')].groupby('segment').agg({'recency': ['mean', 'std'],
                             'frequency': ['mean', 'std', 'sum'],
                             'monetary': ['mean', 'std', 'sum']})

recency             frequency                      monetary                         
                           mean        std       mean        std   sum         mean          std         sum
segment                                                                                                     
at_Risk              165.555000  75.730383   2.001250   0.812141  1601   208.111826  1483.107751  166489.461
new_customers         -2.851528   5.608665  19.834061  23.326030  9084  1236.763876  2705.057945  566437.855
potential_loyalists    9.550765  11.191913   4.180807   1.488346  3006   281.516857   245.196800  202410.620

When we look at the values of the "at_Risk" segment, it is observed that it needs an attention in terms of recency, as expected. In order to attract the attention of the customers in this segment, short-term campaigns can be created in the market areas which customers in this segment are interested in.

Customers in the "new_customers" segment draw attention with being the most revenue-generating and most recent customers. Long-term actions should be taken to reduce the churn rate of these customers. Therefore, the shopping continuity these customers can be increased by organizing campaigns with variable-interval schedules.

Finally, when the "potential_loyalist" segment is looked at, it is observed that it has an balanced structure in terms of both recency and monetary. Therefore, instead of taking action for a specific habit of these people, it is more profitable to focus on the shopping experiences in general of these customers. For example, their shopping habits on our platform can be rewarded by providing a bonus point for every purchase.

# THANK YOU FOR YOUR TIME !!!!!